# Final Project - Movies recommendation

In the final project, we will scrape the text of hollywood movies released since 2000, and build a recommender system based on the textual content of the movies obtained from Wikipedia.

## Phase 1: Gathering data

The wikipedia URLs have the common URL format given below (where XXXX in the below URL format represents the year):


https://en.wikipedia.org/wiki/List_of_American_films_of_XXXX

### Import the required packages

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display # Allows the use of display() for DataFrames
import time
import pickle #To save the objects that were created using webscraping
import pprint
from lxml import html
import requests
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
from urllib.request import urlopen
from bs4 import BeautifulSoup

import urllib
import os
#import io

### Step-1 
Get the URL of each of the movie released since 2000 year from Wikipedia. The following code blocks will get the URLs along with othre details like cast, director, genre and year. The detailes are obtained in 2 pahses. In first phase we get the Movie names, URL and other details for the movies released between 2000 - 2013 years. In the second phase we get the movies details for the years 2014 to 2016. Since the format of the web page is different, we have to do this in 2 phases.

In [60]:
##Get the movies and URLs for the years 2000-2014
URL = list()
Movie_Name = list()
Director = list()
Cast = list()
Genre = list()
year = list()

bs = BeautifulSoup(html)
for y in list(range(2000,2014)):
    url = "https://en.wikipedia.org/wiki/List_of_American_films_of_"+str(y)
    html = urlopen(url)
    time.sleep(3)
    bs = BeautifulSoup(html)
    for table in bs.find_all('table', {"class":"wikitable"}):
        for row in table.find_all('tr'):
            columns = row.find_all('td')
            if len(columns) > 4:
                Movie_Name.append(columns[0].get_text())
                Director.append(columns[1].get_text())
                Cast.append(columns[2].get_text())
                Genre.append(columns[3].get_text())
                year.append(y)
                try:
                    a = columns[0].find('a',href=True)['href']
                    URL.append("https://en.wikipedia.org"+a)
                except:
                    URL.append("NA")
                    continue


                

In [33]:
##Get the movies details for the years 2014 to 2016
URL1 = list()
Movie_Name1 = list()
Director1 = list()
Cast1 = list()
Genre1 = list()
year1 = list()

for y in range(2014,2017):                    
    url = "https://en.wikipedia.org/wiki/List_of_American_films_of_"+str(y)
    try: 
        html = urlopen(url)
    except:
        print("problem with the following URL...continuining...:")
        print(url)
        continue
    time.sleep(3)
    bs = BeautifulSoup(html)
    for table in bs.find_all('table', {"class":"wikitable"}):
        for row in table.find_all('tr'):
            columns = row.find_all('td')
            if len(columns) > 3: #To make sure that we are accessing the movies tables only
                if len(columns) == 6:
                    #print(columns[0].get_text())
                    Movie_Name1.append(columns[0].get_text())
                    Director1.append(columns[1].get_text())
                    Cast1.append(columns[2].get_text())
                    Genre1.append(columns[3].get_text())
                    year1.append(y)
                    try:
                        a=columns[0].find('a',href=True)['href']
                        URL1.append("https://en.wikipedia.org"+a)
                    except:
                        URL1.append("NA")
                        continue
                
                if len(columns) == 7:
                    #print(columns[1].get_text())
                    Movie_Name1.append(columns[1].get_text())
                    Director1.append(columns[2].get_text())
                    Cast1.append(columns[3].get_text())
                    Genre1.append(columns[4].get_text())
                    year1.append(y)
                    
                    try:
                        a=columns[1].find('a',href=True)['href']
                        URL1.append("https://en.wikipedia.org"+a)
                    except:
                        URL1.append("NA")
                        continue

                if len(columns) > 7:
                    #print("col len:{}".format(len(columns)))
                    #print(columns[2].get_text())
                    Movie_Name1.append(columns[2].get_text())
                    Director1.append(columns[3].get_text())
                    Cast1.append(columns[4].get_text())
                    Genre1.append(columns[5].get_text())
                    year1.append(y)                    
                    try:
                        a=columns[2].find('a',href=True)['href']
                        URL1.append("https://en.wikipedia.org"+a)
                    except:
                        URL1.append("NA")
                        continue
                    

## Save the results
We will save the movies details as a CSV file named URL.csv. This helps us to avoid running step-1 again.

In [59]:

df = pd.DataFrame(list(zip(Movie_Name+Movie_Name1,URL+URL1,year+year1,Director+Director1,Cast+Cast1,Genre+Genre1)),\
                  columns=["Movie","URL","Year","Director","Cast","Genre"])

#Remove the rows which do not have URL information

df = df[df["URL"] != "NA"]
#df["Movie_ID"] = range(1,df.shape[0]+1)
#Write the file
df.to_csv("URL.csv",encoding='utf-8',index=False)

ValueError: Unable to coerce to Series, length must be 7: given 487

In [ ]:
##Added later...
URL = pd.read_csv("URL.csv")
cols = URL.columns
URL["Movie_ID"] = URL.index + 1

#print(cols)
display(URL.head())
display(URL.tail())
URL.to_csv("Movies_URL_Latest.csv",encoding='utf-8',index=False)
URL.shape

In [47]:
df["Movie_ID"].max()
#df.head()

4045

Read the saved file to a data frame.

In [48]:
URL = pd.read_csv("URL.csv")

display(URL.head())
display(URL.tail())

URL.shape


,Movie,URL,Year,Director,Cast,Genre,Movie_ID
0,102 Dalmatians,https://en.wikipedia.org/wiki/102_Dalmatians,2000,Kevin Lima,"Glenn Close, Gérard Depardieu, Alice Evans","Comedy, family",1
1,28 Days,https://en.wikipedia.org/wiki/28_Days_(film),2000,Betty Thomas,"Sandra Bullock, Viggo Mortensen",Drama,2
2,3 Strikes,https://en.wikipedia.org/wiki/3_Strikes_(film),2000,DJ Pooh,"Brian Hooks, N'Bushe Wright",Comedy,3
3,The 6th Day,https://en.wikipedia.org/wiki/The_6th_Day,2000,Roger Spottiswoode,"Arnold Schwarzenegger, Robert Duvall",Science fiction,4
4,Across the Line,https://en.wikipedia.org/wiki/Across_the_Line_...,2000,Martin Spottl,"Brad Johnson, Adrienne Barbeau, Brian Bloom",Thriller,5


,Movie,URL,Year,Director,Cast,Genre,Movie_ID
4040,Inferno,https://en.wikipedia.org/wiki/Inferno_(2016_film),2016,Ron Howard,NaN,NaN,4041
4041,Friend Request,https://en.wikipedia.org/wiki/Friend_Request,2016,Simon Verhoeven,Alycia Debnam-Carey,Horror,4042
4042,Rogue One: A Star Wars Story (film),https://en.wikipedia.org/wiki/Rogue_One,2016,Felicity Jones,Diego Luna,Sci-Fi,4043
4043,The Founder,https://en.wikipedia.org/wiki/The_Founder_(film),2016,John Lee Hancock,Michael Keaton,NaN,4044
4044,Rings,https://en.wikipedia.org/wiki/Rings_(2016_film),2016,F. Javier Gutiérrez,NaN,NaN,4045


(4045, 7)

There are 4045 movies that have to be scraped from Wikipedia. Let us do this as batches. Our goal is to scrape the image of the movie (if exists), along with the plot and initial introduction texts.

## Functions
We will code the following functions to obtain the plot information of the movie, along with the release poster image of the movie.

**Open_URL(url)** Gets the HTML content, prepares Beautiful Soup object and returns the Beautiful Soup object. The _url_ parameter represents the complete URL of the webpage.

**Get_Text(bs,file)** Gets the plot of the movie. In case plot is not present, gets the text data of all the HTML Paragraphs of the input Beautiful Soup object (the _bs_) parameter. The _file_ parameter contains the desired file name, which will be the name of the file that contains the movie's plot. Returns -1 or 0 or 1. If -1, a severe error has occurred, and NO text was saved, 0 represents a successful parsing of the plot's text (and the website contains Plot HTML ID), and 1 represents a successful parsion of the complete text (since the web page does NOT have Plot ID).

**Get_Poster(bs, file)** Gets the release poster of the movie and saves the file. Returns 0 if successful else returns -1.





In [30]:
def Open_URL(url):
    '''
    Gets the HTML content, prepares Beautiful Soup object and 
    returns the 
    Beautiful Soup object. 
    The url parameter represents the complete URL of the webpage.
    '''
    try:
         html = urlopen(url)
    except:
        return -1
    try:
        #bs = BeautifulSoup(html).encode("ascii")
        bs = BeautifulSoup(html)
        
        return bs
    except:
        return -2       

def Get_Plot(bs):    
    try:
        p = bs.find("p")
        initial_paragraph = p.getText()
    except:
        return -1

    # collect plot in this list
    plot = []
    
    # find the node with id of "Plot"
    try:
        mark = bs.find(id="Plot")
        # walk through the siblings of the parent (H2) node 
        # until we reach the next H2 node
        for elt in mark.parent.nextSiblingGenerator():
            if elt.name == "h2":
                break
            if hasattr(elt, "text"):
                plot.append(elt.text)
    except:
         return -2
    
    try:
        plot="".join(plot)
        text = initial_paragraph + plot
        return text
    except:
        return -3    
    

    
def Get_All_Text(bs):
    try:
        p = bs.find_all("p")
        l = list()
        for i in p:
            l.append(i.getText())
        return " ".join(l)
    except:
        return -1

def Save_Text_File(text,text_file_name):
    try:
        os.chdir("./data")
    except:
        return -1
    
    with open(text_file_name, 'w',encoding='utf-8') as f:
         f.write(text)
    
    try:
        os.chdir("..")
        return 0
    except:
        return -2

def Get_And_Save_Image(bs,image_file_name):    
    try: 
        img=bs.findAll("img",{"class":"thumbborder"})
        img_URL="https:"+img[0]['src']
    except:
        return -1
     
    try:
        os.chdir("./images")
        ignore=urllib.request.urlretrieve(img_URL,image_file_name)
        os.chdir("..")
        return 0
    except:
        return -2

def Write_Error(url,msg,file):
    with open(file,'a') as f:
        f.write("\n"+msg)
        f.write("\n"+url)

In [57]:
tracker = 0
term=1
start = time.time() # Get start time
print("Beginning the files download...")
#k = 1
for movie, url, year,Movie_ID in zip(list(URL["Movie"]),list(URL["URL"]),list(URL["Year"]),list(URL["Movie_ID"])):
    #if k < 30:
    #    k = k+1
    #    continue
    #print("{},{},{}".format(movie,url,year))
    #Open the URL
    
    bs=Open_URL(url)

    if bs == -1:
        Write_Error(url,"Error in opening the URL","error.txt")
        #print("Error in opening the URL: {}".format(url))
        continue
        

    if bs == -2:
        Write_Error(url,"Error in the creation of bs object for the URL","error.txt")
        #print("Error in the creation of bs object for the URL: {}".format(url))
        continue

    time.sleep(3)

    #create a name for the files
    #image_file_name = str(year)+"_"+movie.strip()+".jpg"
    #text_file_name =  str(year)+"_"+movie.strip()+".txt"
    image_file_name = str(Movie_ID)+".jpg"
    text_file_name =  str(Movie_ID)+".txt"

    text=Get_Plot(bs)
    
    if text == -1:
        Write_Error(url,"No paragraphs are found","error.txt")
        #print("No paragraphs are found")
        #print(url)
        continue

    if text == -2:
        Write_Error(url,"Warning: No Plot ID found","error.txt")
        #print("Warning: No Plot ID found")
        #print(url)
        
        text = Get_All_Text(bs)
        if text == -1:
            Write_Error(url,"No paragraphs are found","error.txt")
            #print("No paragraphs are found")
            #print(url)
            continue
        
    if text == -3:
        Write_Error(url,"Error while appending the main plot with the initial paragraph","error.txt")
        #print("Error while appending the main plot with the initial paragraph")
        #print(url)
        continue
    
    status = Save_Text_File(text,text_file_name)
    
    if status == -1:
        Write_Error(url,"Not able to change the directory to ./data","error.txt")
        #print("Not able to change the directory to ./data")
        #print(url)
        continue
    
    if status == -2:
        Write_Error(url,"Not able to change the directory to .. (parent directory) from ./data","error.txt")
        #print("Not able to change the directory to .. (parent directory) from ./data")
        #print(url)
        continue
        
    #Downloading Image files    
    status = Get_And_Save_Image(bs,image_file_name)
    
    if status == -1:
        Write_Error(url,"Not able to find the image","error.txt")
        #print("Not able to find the image")
        #print(url)
        continue

    if status == -2:
        Write_Error(url,"Not able to save the image","error.txt")
        #print("Not able to save the image")
        #print(url)
        continue

    #Check the status of the webbot    
    tracker = tracker + 1
    if (tracker % 100 == 0):
        print("Processed {} URLs".format(tracker))
        end = time.time() # Get end time
        elapsed_time = end - start
        print("Elapsed time to process 100 URLs:{} secs".format(elapsed_time))
        start = time.time() # Get end time
        #break

    #if term == 1:
    #    break

Beginning the files download...
Processed 100 URLs
Elapsed time to process 100 URLs:446.38204622268677 secs
Processed 200 URLs
Elapsed time to process 100 URLs:433.5029966831207 secs
Processed 300 URLs
Elapsed time to process 100 URLs:461.3205850124359 secs
Processed 400 URLs
Elapsed time to process 100 URLs:458.53809428215027 secs
Processed 500 URLs
Elapsed time to process 100 URLs:476.4255542755127 secs
Processed 600 URLs
Elapsed time to process 100 URLs:473.6112816333771 secs
Processed 700 URLs
Elapsed time to process 100 URLs:458.1378722190857 secs
Processed 800 URLs
Elapsed time to process 100 URLs:434.91878509521484 secs
Processed 900 URLs
Elapsed time to process 100 URLs:569.373973608017 secs
Processed 1000 URLs
Elapsed time to process 100 URLs:533.2878279685974 secs
Processed 1100 URLs
Elapsed time to process 100 URLs:493.94636368751526 secs
Processed 1200 URLs
Elapsed time to process 100 URLs:560.7728536128998 secs
Processed 1300 URLs
Elapsed time to process 100 URLs:519.74306

In [10]:
movie

'https://en.wikipedia.org/wiki/Brother_(2000_film)'

In [ ]:
#Get all text from the URL
getAllText(bs):
    try:
        p = bs.find_all("p")
        for i in p:
             print(i.getText())

In [58]:
tracker = 0
term=1
start = time.time() # Get start time

for movie, url, year in zip(list(URL["Movie"]),list(URL["URL"]),list(URL["Year"])):
    #print("{},{},{}".format(movie,url,year))
    #Open the URL
    
    try:
        html = urlopen(url)
    except:
        print("Not able to connect to the following URL:")
        print(url)
        continue
        
    time.sleep(3)
    #Create a beautiful soup object
    try:
        bs = BeautifulSoup(html)
    except:
        print("Error in the creation of beautifulsoup object")
        print(url)
        continue

    #create a name for the files
    image_file_name = str(year)+"_"+movie.strip()+".jpg"
    text_file_name =  str(year)+"_"+movie.strip()+".txt"
    #print(image_file_name)
    #Get the text (initial paragraph first):
    try:
        p = bs.find("p")
        initial_paragraph = p.getText()
    except:
        print("Error in getting the first paragraph")
        print(url)
        continue
    
    # collect plot in this list
    plot = []
    
    # find the node with id of "Plot"
    try:
        mark = bs.find(id="Plot")
    except:
        print("Error in getting plot marker")
        print(url)
        continue

    # walk through the siblings of the parent (H2) node 
    # until we reach the next H2 node
    try:
        for elt in mark.parent.nextSiblingGenerator():
            if elt.name == "h2":
                break
            if hasattr(elt, "text"):
                plot.append(elt.text)
    except:
        print("Error in extracting plot")
        print(url)
        continue
    
    try:
        plot="".join(plot)
        text = initial_paragraph + plot
    except:
        print("Error in the joining of text")
        print(url)
        continue

    try:
        os.chdir("./data")
    except:
        print("Error while changing the directory to data")
        print(url)
        continue
    
    with open(text_file_name, 'w') as f:
        f.write(text)
    
    try:
        os.chdir("..")
    except:
        print("Error while changing the directory after saving data")
        print(url)
        continue

    
    
    #Downloading Image files
    try: 
        img=bs.findAll("img",{"class":"thumbborder"})
        img_URL="https:"+img[0]['src']
    except:
        print("Error while processing the image file")
        print(img_URL)
        print(url)
    
    try:
        os.chdir("./images")
        #ignore = urllib.request.urlretrieve(img_URL,os.path.basename(image_file_name))
        #urllib.request.urlretrieve(img_URL,os.path.basename(image_file_name))
        ignore=urllib.request.urlretrieve(img_URL,image_file_name)
    except:
        print("Error while changing the directory, to save image file")
        print(img_URL)
        print(url)
        
    try:       
        os.chdir("..")
    except:
        print("Error while changing the directory, after saving the image file")
        print(img_URL)
        print(url)
        
    tracker = tracker + 1
    if (tracker % 10 == 0):
        print("Processed {} URLs".format(tracker))
        end = time.time() # Get end time
        elapsed_time = end - start
        print("Elapsed time to process 100 URLs:{} secs".format(elapsed_time))
        start = time.time() # Get end time
        break

    #if term == 1:
    #    break

2000_102 Dalmatians.jpg
Error in extracting plot
https://en.wikipedia.org/wiki/102_Dalmatians
2000_28 Days.jpg
2000_3 Strikes.jpg
Error in extracting plot
https://en.wikipedia.org/wiki/3_Strikes_(film)
2000_The 6th Day.jpg
Error in extracting plot
https://en.wikipedia.org/wiki/The_6th_Day


KeyboardInterrupt: 

In [2]:
url='https://en.wikipedia.org/wiki/102_Dalmatians'
html = urlopen(url)
bs = BeautifulSoup(html)


In [3]:
p = bs.find_all("p")
l = list()
for i in p:
    l.append(i.getText())
    #print(i.getText())
print(" ".join(l))

102 Dalmatians is a 2000 American family comedy film directed by Kevin Lima in his live-action directorial debut and produced by Edward S. Feldman and Walt Disney Pictures. It is the sequel to the 1996 film 101 Dalmatians and stars Glenn Close reprising her role as Cruella de Vil as she attempts to steal puppies for her "grandest" fur coat yet. Close and Tim McInnerny were the only two actors from the first film to return for the sequel, however. The film was nominated for an Academy Award for Best Costume Design, but lost to Gladiator.[2]   After three years in prison, Cruella de Vil has been cured of her desire for fur coats by Dr. Pavlov and is released into the custody of the probation office on the provision that she will be forced to pay the remainder of her fortune (eight million pounds) to all the dog shelters in the borough of Westminster should she repeat her crime. Cruella therefore mends her working relationship with her valet Alonzo and has him lock away all her fur coats.

In [32]:
p = bs.find("p")
initial_paragraph = p.getText()
print(initial_paragraph)

Rings is a 2017 American supernatural psychological horror film directed by F. Javier Gutiérrez, written by David Loucka, Jacob Aaron Estes and Akiva Goldsman and starring Matilda Lutz, Alex Roe, Johnny Galecki, Vincent D'Onofrio, Aimee Teegarden and Bonnie Morgan. It is the third film in The Ring series and takes place thirteen years after The Ring (2002).


In [39]:
# collect plot in this list
plot = []

# find the node with id of "Plot"
mark = bs.find(id="Plot")

# walk through the siblings of the parent (H2) node 
# until we reach the next H2 node
for elt in mark.parent.nextSiblingGenerator():
    if elt.name == "h2":
        break
    if hasattr(elt, "text"):
        plot.append(elt.text)

# enjoy
plot="".join(plot)
text = initial_paragraph + plot
text

'Rings is a 2017 American supernatural psychological horror film directed by F. Javier Gutiérrez, written by David Loucka, Jacob Aaron Estes and Akiva Goldsman and starring Matilda Lutz, Alex Roe, Johnny Galecki, Vincent D\'Onofrio, Aimee Teegarden and Bonnie Morgan. It is the third film in The Ring series and takes place thirteen years after The Ring (2002).In 2013, on an airplane bound for Seattle, passenger Carter reveals to Faith that he has watched Samara Morgan’s cursed videotape. Faith tells another passenger Kelly, who reveals she has seen the tape too. She asks Carter if he made a copy and after learning he hasn\'t, the airplane begins to malfunction as Samara comes for Carter and eventually causes the plane to crash.Two years later, in 2015, college professor Gabriel buys an old VCR once owned by Carter, discovering the videotape inside. Elsewhere, Julia sees her boyfriend Holt off to college, but becomes concerned when he falls out of contact. She is inspired to find him whe

In [53]:
#https://en.wikipedia.org/wiki/101_Dalmatians_(1996_film)
img=bs.findAll("img",{"class":"thumbborder"})
print(img[0]['src'])

//upload.wikimedia.org/wikipedia/en/thumb/f/fe/102_dalmatians_poster.jpg/220px-102_dalmatians_poster.jpg


In [69]:
img=bs.findAll("",{"class":"image"})
for i in img:
    img_URL = "https://en.wikipedia.org"+i['href']

In [51]:
img=bs.findAll("img",{"class":"thumbborder"})
img_URL="https:"+img[0]['src']
print(img_URL)

https://upload.wikimedia.org/wikipedia/en/thumb/3/30/Digimonthemovie.jpg/220px-Digimonthemovie.jpg


In [39]:
import urllib
import os
os.chdir("./images")
#ignore = urllib.request.urlretrieve(img_URL,os.path.basename("./images/"+img_URL))
ignore = urllib.request.urlretrieve(img_URL,os.path.basename("img_URL.jpg"))
os.chdir("..")

In [52]:
os.chdir("./images")
#ignore = urllib.request.urlretrieve(img_URL,os.path.basename("./images/"+img_URL))
ignore = urllib.request.urlretrieve(img_URL,os.path.basename("img_URL.jpg"))
os.chdir("..")

In [49]:
#url='https://en.wikipedia.org/wiki/Brother_(2000_film)'
#url='https://en.wikipedia.org/wiki/The_Broken_Hearts_Club:_A_Romantic_Comedy'
url='https://en.wikipedia.org/wiki/Digimon:_The_Movie'
html = urlopen(url)
bs = BeautifulSoup(html)
#bs=bs.encode("utf-8")

# collect plot in this list
plot = []

# find the node with id of "Plot"
mark = bs.find(id="Plot")

# walk through the siblings of the parent (H2) node 
# until we reach the next H2 node
for elt in mark.parent.nextSiblingGenerator():
    if elt.name == "h2":
        break
    if hasattr(elt, "text"):
        plot.append(elt.text)

# enjoy
plot="".join(plot)

p = bs.find("p")
initial_paragraph = p.getText()

text = initial_paragraph + plot
text

'Digimon: The Movie is a 2000 American film adaptation of the first three Japanese Digimon films, distributed by 20th Century Fox. The film used footage from the short films Digimon Adventure (1999) and Digimon Adventure: Our War Game! (2000), and the film Digimon Adventure 02: Digimon Hurricane Landing!! / Transcendent Evolution!! The Golden Digimentals (2000).Angela Anaconda[edit]A four-minute short film featuring characters from the Angela Anaconda series appeared before the film in the theatrical and home media releases. In this segment, Angela and her friends line up to watch Digimon: The Movie, but Nannette and her friends cut in line since they have passes. As Angela makes a mad dash to save seats for her friends, Mrs. Brinks (with help from Nannette) sits in front of her, blocking her view. Angela scornfully imagines herself Digivolving into Angelamon to defeat Mrs. Brinks and Nannette, removing all obstacles in enjoying her movie. However, everyone in the audience realizes the

In [50]:
def Save_Text_File(text,text_file_name):
    #try:
    os.chdir("C:/Users/Sekhar/Documents/GitHub/WebAnalytics/Final_Project/data")
    #except:
    #    return -1
    
    with open(text_file_name, 'w',encoding='utf-8') as f:
        f.write(text)
    
    try:
        os.chdir("..")
        return 0
    except:
        return -2


In [29]:
Save_Text_File(text,"temp")

0

In [52]:
df[df["Movie"]=="Digimon"]

,Movie,URL,Year,Director,Cast,Genre,Movie_ID


In [55]:
df.iloc[31:50,]

,Movie,URL,Year,Director,Cast,Genre,Movie_ID
32,Bring It On,https://en.wikipedia.org/wiki/Bring_It_On_(film),2000,Peyton Reed,"Kirsten Dunst, Eliza Dushku, Gabrielle Union, ...",Comedy,32
33,The Broken Hearts Club: A Romantic Comedy,https://en.wikipedia.org/wiki/The_Broken_Heart...,2000,Greg Berlanti,"Ben Weber, Timothy Olyphant, Zach Braff",Comedy,33
34,Brother,https://en.wikipedia.org/wiki/Brother_(2000_film),2000,Takeshi Kitano,"Takeshi Kitano, Omar Epps",Crime drama,34
35,Cast Away,https://en.wikipedia.org/wiki/Cast_Away,2000,Robert Zemeckis,"Tom Hanks, Helen Hunt",Adventure,35
36,Cecil B. Demented,https://en.wikipedia.org/wiki/Cecil_B._Demented,2000,John Waters,"Melanie Griffith, Stephen Dorff, Alicia Witt, ...",Comedy,36
37,The Cell,https://en.wikipedia.org/wiki/The_Cell,2000,Tarsem Singh,"Jennifer Lopez, Vince Vaughn, Vincent D'Onofrio",Science fiction,37
38,Center Stage,https://en.wikipedia.org/wiki/Center_Stage_(20...,2000,Nicholas Hytner,"Amanda Schull, Peter Gallagher, Ethan Stiefel",Musical,38
39,Charlie's Angels,https://en.wikipedia.org/wiki/Charlie%27s_Ange...,2000,McG,"Drew Barrymore, Cameron Diaz, Lucy Liu, Bill M...",Action,39
40,Cherry Falls,https://en.wikipedia.org/wiki/Cherry_Falls,2000,Geoffrey Wright,"Brittany Murphy, Michael Biehn, Jesse Bradford",Thriller,40
41,Chinese Coffee,https://en.wikipedia.org/wiki/Chinese_Coffee,2000,Al Pacino,"Al Pacino, Jerry Orbach",Drama,41


In [ ]:
file_names = os.listdir("./data")
file_names = [i for i in file_names if re.search('[1-9]*\.txt',i)]
y = list()
x = list()
k = 0
for i in file_names:
    y.append(int(i.split(".")[0]))
    #print(y)
    tokens = Read_File("./data/"+i)
    tokens = Remove_Stop_Words(tokens)
    cleaned_text = Clean_Text(tokens)
    x.append(cleaned_text)
    k = k+1
    if(k%100 == 0):
        print("Processed {} files".format(k))
    

In [66]:
#Finding the missing files
import re
file_names = os.listdir("./data")
file_names = [i for i in file_names if re.search('[1-9]*\.txt',i)]
y = list()
for i in file_names:
    y.append(int(i.split(".")[0]))
y = set(y)
x = set(range(1,4046))
print(x - y)
missing = list(x - y)
print(missing)

{800, 1344, 3752, 2859, 493, 3664, 2745, 2302}
[800, 1344, 3752, 2859, 493, 3664, 2745, 2302]


In [70]:
tracker = 0
term=1
start = time.time() # Get start time
print("Beginning the files download...")
#k = 1
for movie, url, year,Movie_ID in zip(list(df["Movie"]),list(df["URL"]),list(df["Year"]),list(df["Movie_ID"])):
    if Movie_ID in missing:
        print("processing {}".format(Movie_ID))
        continue
    
    bs=Open_URL(url)

    if bs == -1:
        Write_Error(url,"Error in opening the URL","error.txt")
        #print("Error in opening the URL: {}".format(url))
        continue
        

    if bs == -2:
        Write_Error(url,"Error in the creation of bs object for the URL","error.txt")
        #print("Error in the creation of bs object for the URL: {}".format(url))
        continue

    time.sleep(3)

    #create a name for the files
    #image_file_name = str(year)+"_"+movie.strip()+".jpg"
    #text_file_name =  str(year)+"_"+movie.strip()+".txt"
    image_file_name = str(Movie_ID)+".jpg"
    text_file_name =  str(Movie_ID)+".txt"

    text=Get_Plot(bs)
    
    if text == -1:
        Write_Error(url,"No paragraphs are found","error.txt")
        #print("No paragraphs are found")
        #print(url)
        continue

    if text == -2:
        Write_Error(url,"Warning: No Plot ID found","error.txt")
        #print("Warning: No Plot ID found")
        #print(url)
        
        text = Get_All_Text(bs)
        if text == -1:
            Write_Error(url,"No paragraphs are found","error.txt")
            #print("No paragraphs are found")
            #print(url)
            continue
        
    if text == -3:
        Write_Error(url,"Error while appending the main plot with the initial paragraph","error.txt")
        #print("Error while appending the main plot with the initial paragraph")
        #print(url)
        continue
    
    status = Save_Text_File(text,text_file_name)
    
    if status == -1:
        Write_Error(url,"Not able to change the directory to ./data","error.txt")
        #print("Not able to change the directory to ./data")
        #print(url)
        continue
    
    if status == -2:
        Write_Error(url,"Not able to change the directory to .. (parent directory) from ./data","error.txt")
        #print("Not able to change the directory to .. (parent directory) from ./data")
        #print(url)
        continue
        
    #Downloading Image files    
    status = Get_And_Save_Image(bs,image_file_name)
    
    if status == -1:
        Write_Error(url,"Not able to find the image","error.txt")
        #print("Not able to find the image")
        #print(url)
        continue

    if status == -2:
        Write_Error(url,"Not able to save the image","error.txt")
        #print("Not able to save the image")
        #print(url)
        continue

    #Check the status of the webbot    
    tracker = tracker + 1
    if (tracker % 100 == 0):
        print("Processed {} URLs".format(tracker))
        end = time.time() # Get end time
        elapsed_time = end - start
        print("Elapsed time to process 100 URLs:{} secs".format(elapsed_time))
        start = time.time() # Get end time
        #break

    #if term == 1:
    #    break

Beginning the files download...


KeyboardInterrupt: 

In [78]:
df[df["Movie_ID"]==800]
os.chdir("..")

In [96]:
##Code to download the missing images:




#Finding the missing images
import re
image_files = os.listdir("./images")
text_files = os.listdir("./data")

#Make sure that we are dealing with movie data files only
image_files = [i for i in image_files if re.search('[1-9]*\.jpg',i)]
text_files  = [i for i in text_files if re.search('[1-9]*\.txt',i)]

y = list()
x = list()
for i in image_files:
    y.append(int(i.split(".")[0]))
y = set(y)

for i in text_files:
    x.append(int(i.split(".")[0]))
missing_image_ids = list(set(x) - set(y))
len(missing_image_ids)+3749

4037

In [97]:
missing_images_df = df[df["Movie_ID"].isin(missing_image_ids)]
missing_images_df

,Movie,URL,Year,Director,Cast,Genre,Movie_ID
47,The Crossing,https://en.wikipedia.org/wiki/The_Crossing_(20...,2000,Robert Harmon,"Jeff Daniels, Roger Rees, Sebastian Roché, Ste...",Historical Drama,47
61,The Eyes of Tammy Faye,https://en.wikipedia.org/wiki/The_Eyes_of_Tamm...,2000,"Fenton Bailey, Randy Barbato","Tammy Faye Bakker, RuPaul",Documentary,61
64,Final Destination,https://en.wikipedia.org/wiki/Final_Destination,2000,James Wong,"Devon Sawa, Ali Larter, Kerr Smith, Seann Will...",Horror,64
89,Icebreaker,https://en.wikipedia.org/wiki/Icebreaker_(film),2000,David Giancola,"Sean Astin, Stacy Keach, Bruce Campbell",Action,89
104,Long Night's Journey into Day,https://en.wikipedia.org/wiki/Long_Night%27s_J...,2000,"Deborah Hoffmann, Frances Reid",the Truth and Reconciliation Commission in pos...,Documentary,104
109,Maryam,https://en.wikipedia.org/wiki/Maryam_(film),2000,Ramin Serry,"Mariam Parris, Shaun Toub, Shohreh Aghdashloo",Drama,109
134,Pay It Forward,https://en.wikipedia.org/wiki/Pay_It_Forward_(...,2000,Mimi Leder,"Haley Joel Osment, Helen Hunt, Kevin Spacey, J...",Drama,134
183,Thirteen Days,https://en.wikipedia.org/wiki/Thirteen_Days_(f...,2000,Roger Donaldson,"Kevin Costner, Bruce Greenwood, Dylan Baker, S...",Drama,182
229,Amy's Orgasm,https://en.wikipedia.org/wiki/Amy%27s_Orgasm,2001,Julie Davis,"Julie Davis, Nick Chinlund, Jeff Cesario",Comedy,227
236,Bandits,https://en.wikipedia.org/wiki/Bandits,2001,Barry Levinson,"Bruce Willis, Cate Blanchett, Billy Bob Thornton","Crime, Comedy",234


In [94]:
for url, Movie_ID in zip(list(missing_images_df["URL"]),list(missing_images_df["Movie_ID"])):
    #print(URL)
    try:
        html = urlopen(url)
    except:
        print("Not able to connect to the following URL:")
        print(url)
        continue
        
    time.sleep(3)
    image_file_name = str(Movie_ID)+".jpg"
    #Create a beautiful soup object
    try:
        bs = BeautifulSoup(html)
    except:
        print("Error in the creation of beautifulsoup object")
        print(url)
        continue

        
    try: 
        img=bs.findAll("img",{"class":"thumbborder"})
        img_URL="https:"+img[0]['src']
        print(img_URL)
    except:
        print(url)
        print(Movie_ID)
        print("NOT found in thumbborder..checking alternative path")
        try: 
            img=bs.findAll(" ",{"class":"image"})
            img_URL="https:"+img[0]['href']
            print(img_URL)
            print("Found at alternative path")
        except:    
            print("Not found at alternative path also")
            continue
     
    try:
        os.chdir("./images")
        ignore=urllib.request.urlretrieve(img_URL,image_file_name)
        os.chdir("..")
        print("successfuly downloaded")
    except:
        print("Not able to change the directory")
        continue
        


https://en.wikipedia.org/wiki/The_Crossing_(2000_film)
47
NOT found in thumbborder..checking alternative path
Not found at alternative path also
https://en.wikipedia.org/wiki/The_Eyes_of_Tammy_Faye
61
NOT found in thumbborder..checking alternative path
Not found at alternative path also


KeyboardInterrupt: 

In [91]:
os.chdir("..")

In [142]:
url='https://en.wikipedia.org/wiki/The_Crossing_(2000_film)'

html = urlopen(url)
img=bs.findAll("meta")
for i in img:
    print(i)
#print(img)
#img_URL="https://en.wikipedia.org"+img[0]['href']
#print(img_URL)


<meta charset="utf-8"/>
<meta content="" name="ResourceLoaderDynamicStyles"/>
<meta content="MediaWiki 1.30.0-wmf.7" name="generator"/>
<meta content="origin-when-cross-origin" name="referrer"/>


In [160]:
url='https://en.wikipedia.org/wiki/The_Crossing_(2000_film)'

html = urlopen(url)

s = str(html.read())
#s.split(" ")
s = [i for i in s.split() if re.search('https:.*\.jpg',i)]
s = s[0].split('"/>')
s = s[0].split()

['content="https://upload.wikimedia.org/wikipedia/en/2/2a/DVD_cover_of_the_movie_The_Crossing.jpg',
 '\\n<link']

In [176]:
html = urlopen(url)

s = str(html.read())
s = [i for i in s.split() if re.search('https:.*\.jpg',i)][0]
s=s.split("https:")[1]
s=s.split('"')[0]
s


'//upload.wikimedia.org/wikipedia/en/2/2a/DVD_cover_of_the_movie_The_Crossing.jpg'

In [177]:
for url, Movie_ID in zip(list(missing_images_df["URL"]),list(missing_images_df["Movie_ID"])):
    html = urlopen(url)

    s = str(html.read())
    s = [i for i in s.split() if re.search('https:.*\.jpg',i)][0]
    s=s.split("https:")[1]
    s=s.split('"')[0]
    print(s)

//upload.wikimedia.org/wikipedia/en/2/2a/DVD_cover_of_the_movie_The_Crossing.jpg


IndexError: list index out of range